In [1]:
from __future__ import unicode_literals, print_function, division

In [2]:
from io import open
import glob

def findFiles(path): return glob.glob(path)

print(findFiles('data/names/*.txt'))

import unicodedata
import string

all_letters = string.ascii_letters + " .,;'"
n_letters = len(all_letters)


['data/names/Czech.txt', 'data/names/German.txt', 'data/names/Arabic.txt', 'data/names/Japanese.txt', 'data/names/Chinese.txt', 'data/names/Vietnamese.txt', 'data/names/Russian.txt', 'data/names/French.txt', 'data/names/Irish.txt', 'data/names/English.txt', 'data/names/Spanish.txt', 'data/names/Greek.txt', 'data/names/Italian.txt', 'data/names/Portuguese.txt', 'data/names/Scottish.txt', 'data/names/Dutch.txt', 'data/names/Korean.txt', 'data/names/Polish.txt']


In [3]:
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
        and c in all_letters
    )

print(unicodeToAscii('Ślusàrski'))

Slusarski


In [4]:
category_lines = {}
all_categories = []

# Read a file and split into lines
def readLines(filename):
    lines = open(filename, encoding='utf-8').read().strip().split('\n')
    return [unicodeToAscii(line) for line in lines]

In [5]:
for filename in findFiles('data/names/*.txt'):
    category = filename.split('/')[-1].split('.')[0]
    all_categories.append(category)
    lines = readLines(filename)
    category_lines[category] = lines

n_categories = len(all_categories)

In [6]:
all_categories


['Czech',
 'German',
 'Arabic',
 'Japanese',
 'Chinese',
 'Vietnamese',
 'Russian',
 'French',
 'Irish',
 'English',
 'Spanish',
 'Greek',
 'Italian',
 'Portuguese',
 'Scottish',
 'Dutch',
 'Korean',
 'Polish']

In [7]:
category_lines['Czech']

['Abl',
 'Adsit',
 'Ajdrna',
 'Alt',
 'Antonowitsch',
 'Antonowitz',
 'Bacon',
 'Ballalatak',
 'Ballaltick',
 'Bartonova',
 'Bastl',
 'Baroch',
 'Benesch',
 'Betlach',
 'Biganska',
 'Bilek',
 'Blahut',
 'Blazek',
 'Blazek',
 'Blazejovsky',
 'Blecha',
 'Bleskan',
 'Blober',
 'Bock',
 'Bohac',
 'Bohunovsky',
 'Bolcar',
 'Borovka',
 'Borovski',
 'Borowski',
 'Borovsky',
 'Brabbery',
 'Brezovjak',
 'Brousil',
 'Bruckner',
 'Buchta',
 'Cablikova',
 'Camfrlova',
 'Cap',
 'Cerda',
 'Cermak',
 'Chermak',
 'Cermak',
 'Cernochova',
 'Cernohous',
 'Cerny',
 'Cerney',
 'Cerny',
 'Cerv',
 'Cervenka',
 'Chalupka',
 'Charlott',
 'Chemlik',
 'Chicken',
 'Chilar',
 'Chromy',
 'Cihak',
 'Clineburg',
 'Klineberg',
 'Cober',
 'Colling',
 'Cvacek',
 'Czabal',
 'Damell',
 'Demall',
 'Dehmel',
 'Dana',
 'Dejmal',
 'Dempko',
 'Demko',
 'Dinko',
 'Divoky',
 'Dolejsi',
 'Dolezal',
 'Doljs',
 'Dopita',
 'Drassal',
 'Driml',
 'Duyava',
 'Dvorak',
 'Dziadik',
 'Egr',
 'Entler',
 'Faltysek',
 'Faltejsek',
 'Fencl',

In [8]:
import torch

# Find letter index from all_letters, e.g. "a" = 0
def letterToIndex(letter):
    return all_letters.find(letter)

# Just for demonstration, turn a letter into a <1 x n_letters> Tensor
def letterToTensor(letter):
    tensor = torch.zeros(1, n_letters)
    tensor[0][letterToIndex(letter)] = 1
    return tensor

# Turn a line into a <line_length x 1 x n_letters>,
# or an array of one-hot letter vectors
def lineToTensor(line):
    tensor = torch.zeros(len(line), 1, n_letters)
    for li, letter in enumerate(line):
        tensor[li][0][letterToIndex(letter)] = 1
    return tensor

print(letterToTensor('J'))

print(lineToTensor('Jones').size())



Columns 0 to 12 
    0     0     0     0     0     0     0     0     0     0     0     0     0

Columns 13 to 25 
    0     0     0     0     0     0     0     0     0     0     0     0     0

Columns 26 to 38 
    0     0     0     0     0     0     0     0     0     1     0     0     0

Columns 39 to 51 
    0     0     0     0     0     0     0     0     0     0     0     0     0

Columns 52 to 56 
    0     0     0     0     0
[torch.FloatTensor of size 1x57]

torch.Size([5, 1, 57])


In [10]:
import torch.nn as nn
from torch.autograd import Variable

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()

        self.hidden_size = hidden_size

        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(input_size + hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        combined = torch.cat((input, hidden), 1)
        hidden = self.i2h(combined)
        output = self.i2o(combined)
        output = self.softmax(output)
        return output, hidden

    def initHidden(self):
        return Variable(torch.zeros(1, self.hidden_size))

n_hidden = 128
rnn = RNN(n_letters, n_hidden, n_categories)

In [11]:
n_letters

57

In [12]:
n_categories

18

In [13]:
input = Variable(lineToTensor('Albert'))
hidden = Variable(torch.zeros(1, n_hidden))

output, next_hidden = rnn(input[0], hidden)
print(output)

Variable containing:

Columns 0 to 9 
-2.8980 -3.0456 -2.8910 -2.8496 -2.8043 -2.9891 -2.9457 -2.8158 -2.8486 -2.8312

Columns 10 to 17 
-2.9021 -2.8805 -2.8748 -2.8604 -2.8268 -2.9190 -2.9150 -2.9632
[torch.FloatTensor of size 1x18]



In [14]:
import random

def randomChoice(l):
    return l[random.randint(0, len(l) - 1)]

def randomTrainingExample():
    category = randomChoice(all_categories)
    line = randomChoice(category_lines[category])
    category_tensor = Variable(torch.LongTensor([all_categories.index(category)]))
    line_tensor = Variable(lineToTensor(line))
    return category, line, category_tensor, line_tensor

for i in range(10):
    category, line, category_tensor, line_tensor = randomTrainingExample()
    print('category =', category, '/ line =', line)

category = Scottish / line = Cunningham
category = Korean / line = Rheem
category = French / line = Boucher
category = Spanish / line = Rodriguez
category = Italian / line = Franzese
category = Portuguese / line = Araullo
category = Korean / line = Choi
category = German / line = Worner
category = Japanese / line = Minatoya
category = English / line = Blackwell


In [15]:
criterion = nn.NLLLoss()

In [16]:
learning_rate = 0.005 # If you set this too high, it might explode. If too low, it might not learn

def train(category_tensor, line_tensor):
    hidden = rnn.initHidden()

    rnn.zero_grad()

    for i in range(line_tensor.size()[0]):
        output, hidden = rnn(line_tensor[i], hidden)

    loss = criterion(output, category_tensor)
    loss.backward()

    # Add parameters' gradients to their values, multiplied by learning rate
    for p in rnn.parameters():
        p.data.add_(-learning_rate, p.grad.data)

    return output, loss.data[0]

In [19]:
import time
import math

n_iters = 100000
print_every = 5000
plot_every = 1000



# Keep track of losses for plotting
current_loss = 0
all_losses = []

def timeSince(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

start = time.time()

for iter in range(1, n_iters + 1):
    category, line, category_tensor, line_tensor = randomTrainingExample()
    output, loss = train(category_tensor, line_tensor)
    current_loss += loss

    # Print iter number, loss, name and guess
    if iter % print_every == 0:
        guess, guess_i = categoryFromOutput(output)
        correct = '✓' if guess == category else '✗ (%s)' % category
        print('%d %d%% (%s) %.4f %s / %s %s' % (iter, iter / n_iters * 100, timeSince(start), loss, line, guess, correct))

    # Add current loss avg to list of losses
    if iter % plot_every == 0:
        all_losses.append(current_loss / plot_every)
        current_loss = 0

5000 5% (0m 3s) 2.7563 Rademakers / Greek ✗ (Dutch)
10000 10% (0m 7s) 3.7581 Hay / Chinese ✗ (Scottish)
15000 15% (0m 11s) 1.9887 Favre / French ✓
20000 20% (0m 14s) 1.2311 Srour / Arabic ✓
25000 25% (0m 18s) 0.4511 Snyders / Dutch ✓
30000 30% (0m 22s) 1.2200 Mckay / Scottish ✓
35000 35% (0m 25s) 1.3520 Janick / Czech ✓
40000 40% (0m 29s) 2.5827 Flynn / Scottish ✗ (Irish)
45000 45% (0m 33s) 2.6357 Stamp / Arabic ✗ (English)
50000 50% (0m 36s) 1.3493 Yun / Chinese ✗ (Korean)
55000 55% (0m 40s) 2.7112 Prill / English ✗ (Czech)
60000 60% (0m 44s) 2.2078 Said / Korean ✗ (Arabic)
65000 65% (0m 47s) 3.4021 StrakaO / Arabic ✗ (Czech)
70000 70% (0m 51s) 2.5285 Hall / Scottish ✗ (German)
75000 75% (0m 55s) 2.2128 Hase / Japanese ✗ (German)
80000 80% (0m 58s) 0.4362 Henriques / Portuguese ✓
85000 85% (1m 2s) 0.0656 Cathain / Irish ✓
90000 90% (1m 6s) 0.7203 D'cruze / Portuguese ✓
95000 95% (1m 9s) 2.6893 Wizner / Polish ✗ (Czech)
100000 100% (1m 13s) 2.4736 Bell / German ✗ (Scottish)


In [18]:
def categoryFromOutput(output):
    top_n, top_i = output.data.topk(1) # Tensor out of Variable with .data
    category_i = top_i[0][0]
    return all_categories[category_i], category_i